In [62]:
import os
import csv
import numpy
import pandas
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [19]:
csv_file = 'sensor_data_labeled.csv'
df = pandas.read_csv(csv_file)

df = df.drop(columns=['datetime', 'epoch', 'humidity', 'pressure'])
df['voc'] = pandas.to_numeric(df['voc'], errors='coerce')
df = df.dropna(subset=['voc'])
df['voc'] = df['voc'].astype(int)
df.head()


,dust,mq135,mq7,temperature,voc,label
0,21,5006,3357,28.55,139,normal
1,25,5012,3356,28.55,138,normal
2,18,5009,3357,28.55,139,normal
3,26,5002,3352,28.56,139,normal
4,23,4997,3347,28.54,127,normal


In [21]:
X = df[['dust', 'mq135', 'mq7', 'temperature', 'voc']].values

y_labels = df['label'].values

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y_labels)
y_categorical = to_categorical(y_encoded)



In [83]:
print(len(y_categorical))
print(y_categorical[2600])

5999
[1. 0. 0. 0.]


In [86]:
categories = ['aktivitas','kimia','makanminum', 'normal']

In [84]:
model = Sequential([
    Dense(64, input_shape=(5,), activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(4, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X, y_categorical, epochs=100, batch_size=4, verbose=1, validation_split=0.4)


Epoch 1/100


c:\Users\atmatech\Documents\python_pred_esp32\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


900/900 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7703 - loss: 0.6541 - val_accuracy: 0.4496 - val_loss: 2.4608
Epoch 2/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9022 - loss: 0.2981 - val_accuracy: 0.4496 - val_loss: 3.6231
Epoch 3/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8980 - loss: 0.2871 - val_accuracy: 0.4442 - val_loss: 4.2574
Epoch 4/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9040 - loss: 0.2815 - val_accuracy: 0.4483 - val_loss: 4.5684
Epoch 5/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9028 - loss: 0.2776 - val_accuracy: 0.4467 - val_loss: 4.8081
Epoch 6/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9149 - loss: 0.2582 - val_accuracy: 0.4458 - val_loss: 4.9555
Epoch 7/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9148 - loss: 0.2464 - val_accuracy: 0.4492 - val_loss: 4.6912
Epoch 8/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9287 - loss: 0.2077 - val_accuracy: 0.4467

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], 'b', label='Training accuracy')
plt.plot(history.history['val_accuracy'], 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], 'b', label='Training loss')
plt.plot(history.history['val_loss'], 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [87]:
model.save('model_feedforward.h5')

In [91]:
id = 4500
print(X[id])
print(y_labels[id])

data_to_predict = numpy.array([[  24. ,  5949.  , 4714.,28.77, 94.]]) 
y_pred = model.predict(data_to_predict)

predicted_class = numpy.argmax(y_pred, axis=1)
print(f"Prediksi kelas: {categories[predicted_class[0]]} >> {[predicted_class[0]]}")


[  24.   5949.   4714.     28.77   94.  ]
kimia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Prediksi kelas: aktivitas >> [np.int64(0)]


In [92]:
loss, accuracy = model.evaluate(X, y_categorical, verbose=0)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

Loss: 9.1409, Accuracy: 0.7496


In [93]:
converter = tensorflow.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model_feedforward.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\atmatech\AppData\Local\Temp\tmpmyopqw8p\assets


INFO:tensorflow:Assets written to: C:\Users\atmatech\AppData\Local\Temp\tmpmyopqw8p\assets


Saved artifact at 'C:\Users\atmatech\AppData\Local\Temp\tmpmyopqw8p'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 5), dtype=tf.float32, name='keras_tensor_125')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2102102466704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2102102469776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2102102470160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2102102469392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2102102469200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2102102469968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2102102470928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2102102472080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2102102472464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2102102470544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  21021024